## Wczytywanie tekstu

In [1]:
import urllib.request
import tiktoken

In [3]:
url = ("https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x233d43ed310>)

In [4]:
with open("the-verdict-pl.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Łączna liczba znaków: {len(raw_text)}")
print(raw_text[:99])

Łączna liczba znaków: 20533
Zawsze uważałem Jacka Gisburna za raczej taniego geniusza – choć skądinąd poczciwego faceta – nie b


In [5]:
#Prosty podział tekstu z regex
import re

text = "Witaj świecie. To jest test."
result = re.split(r"(\s)", text)
print(result)

['Witaj', ' ', 'świecie.', ' ', 'To', ' ', 'jest', ' ', 'test.']


In [6]:
result = re.split(r"([,.]|\s)", text)
print(result)

['Witaj', ' ', 'świecie', '.', '', ' ', 'To', ' ', 'jest', ' ', 'test', '.', '']


In [7]:
#usunięcie białych znaków
result = [item for item in result if item.strip()]
print(result)

['Witaj', 'świecie', '.', 'To', 'jest', 'test', '.']


In [8]:
#Pełny regex filtrujący białe i specjalne znaki

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

3770


In [9]:
print(preprocessed[:30])

['Zawsze', 'uważałem', 'Jacka', 'Gisburna', 'za', 'raczej', 'taniego', 'geniusza', '–', 'choć', 'skądinąd', 'poczciwego', 'faceta', '–', 'nie', 'było', 'więc', 'dla', 'mnie', 'wielkim', 'zaskoczeniem', ',', 'gdy', 'dowiedziałem', 'się', ',', 'że', 'u', 'szczytu', 'sławy']


In [10]:
all_worlds = sorted(set(preprocessed))
vocab_size = len(all_worlds)
print(vocab_size)

1590


In [11]:
all_worlds[:10]

['!', '(', ')', ',', '.', ':', ';', '?', 'A', 'Ach']

In [12]:
vocab = {token:integer for integer, token in enumerate(all_worlds)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('(', 1)
(')', 2)
(',', 3)
('.', 4)
(':', 5)
(';', 6)
('?', 7)
('A', 8)
('Ach', 9)
('Ale', 10)
('Ani', 11)
('Biedna', 12)
('Biedny', 13)
('Bywały', 14)
('Być', 15)
('Był', 16)
('Było', 17)
('Bóg', 18)
('Carlo', 19)
('Chciała', 20)
('Chciałbym', 21)
('Chicago', 22)
('Chodź', 23)
('Choć', 24)
('Ciebie', 25)
('Claude', 26)
('Co', 27)
('Croft', 28)
('Czy', 29)
('Czyż', 30)
('Człowiek', 31)
('Cóż', 32)
('Devonshire', 33)
('Dlaczego', 34)
('Do', 35)
('Dopiero', 36)
('Dubarry', 37)
('Florencji', 38)
('Gallery', 39)
('Gdy', 40)
('Gdybym', 41)
('Gideon', 42)
('Gisburn', 43)
('Gisburna', 44)
('Grafton', 45)
('Grindle', 46)
('Grindle’a', 47)
('Grindle’u”', 48)
('Hermia', 49)
('Hermii', 50)


## Budowa tokenizera

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)
text = """To ostatni, jaki namalował powiedziała pani Gisburn z wybaczalną dumą."""
ids = tokenizer.encode(text)
print(ids)

[147, 814, 3, 473, 684, 933, 835, 43, 1428, 1370, 370, 4]


In [15]:
print(tokenizer.decode(ids))

To ostatni, jaki namalował powiedziała pani Gisburn z wybaczalną dumą.


In [9]:
# test na próbce, której nie ma w danych
text = "Witaj, czy lubisz kawę?"
#print(tokenizer.encode(text))

In [17]:
all_tokens = sorted(list(set(preprocessed)))
preprocessed[:20]

['Zawsze',
 'uważałem',
 'Jacka',
 'Gisburna',
 'za',
 'raczej',
 'taniego',
 'geniusza',
 '–',
 'choć',
 'skądinąd',
 'poczciwego',
 'faceta',
 '–',
 'nie',
 'było',
 'więc',
 'dla',
 'mnie',
 'wielkim']

In [18]:
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1592


In [19]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('„temat”', 1587)
('„zabójcza', 1588)
('„zrobiony”', 1589)
('<|endoftext|>', 1590)
('<|unk|>', 1591)


In [20]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [21]:
text1 = "Witaj, czy lubisz kawę?"
text2 = "Wyszedł na skąpany w słońcu taras pałacu"
text = " <|endoftext|> ".join((text1, text2))
print(text)

Witaj, czy lubisz kawę? <|endoftext|> Wyszedł na skąpany w słońcu taras pałacu


In [22]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1591, 3, 306, 576, 1591, 7, 1590, 1591, 664, 1591, 1312, 1591, 1239, 1591]


In [23]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, czy lubisz <|unk|>? <|endoftext|> <|unk|> na <|unk|> w <|unk|> taras <|unk|>


Inne symbole specjalne:
[BOS] - beginning of sequence - początek tesktu/sekwencji
[EOS] - end of sequence - koniec sekwenci/tekstu
[PAD] - padding - przedłużanie tekstów 

### Kodowanie par bajtów

Algorytm BPE znajduje najczęściej pojawiające się fragmenty i łaczy je w większe jednostki. 
Rozdziela słowo na pojedyncze znaki, algorytm sprawdza, które pary wsytępują najczęsciej w całym tekście. Pary zostają połączone w token. Proces jest powtarzany. Dzięki czemu każde słowo, to nie osoby token

In [24]:
tokenizer = tiktoken.get_encoding("gpt2")
text = ("Cześć, lubisz herbatę? <|endoftext|> Na skąpany w słońcu taras "
"jakiegoś nieznanegoMiejsca.")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[34, 2736, 129, 249, 38325, 11, 300, 46676, 89, 607, 8664, 128, 247, 30, 220, 50256, 11013, 1341, 128, 227, 79, 1092, 266, 264, 41615, 78, 129, 226, 27399, 13422, 292, 474, 461, 494, 2188, 129, 249, 299, 494, 47347, 1531, 2188, 44, 494, 73, 1416, 64, 13]


In [25]:
strings = tokenizer.decode(integers)
print(strings)

Cześć, lubisz herbatę? <|endoftext|> Na skąpany w słońcu taras jakiegoś nieznanegoMiejsca.


In [ ]:
# Test tokenizerów na trudnym polskim wyrazie

text = "Żółć gęślą jaźń"
enc_gpt2 = tiktoken.get_encoding("gpt2")
enc_gpt4 = tiktoken.get_encoding("cl100k_base")

print(f"GPT2: {len(enc_gpt2.encode(text))}")
print(f"GPT4: {len(enc_gpt4.encode(text))}")

GPT2: 18
GPT4: 12


### Próbkowanie danych

In [2]:
with open("the-verdict-pl.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("cl100k_base")
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

7831


#### Przykład demonstracyjny

In [3]:
enc_sample = enc_text[50:]
context_size = 4 # określa liczbę tokenów w wejściu
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(x)
print(y)

[57587, 1142, 70217, 336]
[1142, 70217, 336, 12951]


In [4]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[57587] ----> 1142
[57587, 1142] ----> 70217
[57587, 1142, 70217] ----> 336
[57587, 1142, 70217, 336] ----> 12951


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 dow ----> ied
 dowied ----> ział
 dowiedział ----> em
 dowiedziałem ---->  się


### Komponent ładujący dane

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_lenght, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # tokenizuje tekst

        for i in range(0, len(token_ids) - max_lenght, stride): # okno kontekstowe dzielace tekst na nakładające sie sekwencje
            input_chunk = token_ids[i:i + max_lenght]
            target_chunk = token_ids[i + 1: i + max_lenght + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

#### Ładowanie danych

In [6]:
def create_dataloader_v1(txt, batch_size=4, max_lenght=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    dataset = GPTDatasetV1(txt, tokenizer, max_lenght, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [7]:
with open("the-verdict-pl.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()


datalaoder = create_dataloader_v1(raw_text, batch_size=1, max_lenght=8, stride=2, shuffle=False)
data_iter = iter(datalaoder)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   57,  8805,  3059,   577, 10196,  6077,    64, 75432]]), tensor([[ 8805,  3059,   577, 10196,  6077,    64, 75432,  7762]])]


In [9]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 3059,   577, 10196,  6077,    64, 75432,  7762,    64]]), tensor([[  577, 10196,  6077,    64, 75432,  7762,    64,   480]])]


In [11]:
datalaoder = create_dataloader_v1(
    raw_text,
    batch_size=8,
    max_lenght=4,
    stride=4,
    shuffle=False
)

data_iter = iter(datalaoder)
inputs, targets = next(data_iter)
print("Wejścia:\n", inputs)
print("\nCele:\n", targets)

Wejścia:
 tensor([[   57,  8805,  3059,   577],
        [10196,  6077,    64, 75432],
        [ 7762,    64,   480,   285],
        [22464,    64, 15036,  9148],
        [46894, 14531, 63387, 35201],
        [ 4458,  1389,  2665,  7886],
        [ 1940, 62151,   258, 62151],
        [57403,    89,  5979,    86]])

Cele:
 tensor([[ 8805,  3059,   577, 10196],
        [ 6077,    64, 75432,  7762],
        [   64,   480,   285, 22464],
        [   64, 15036,  9148, 46894],
        [14531, 63387, 35201,  4458],
        [ 1389,  2665,  7886,  1940],
        [62151,   258, 62151, 57403],
        [   89,  5979,    86, 13546]])


## Osadzenia tokenów

In [13]:
inputs_ids = torch.tensor([2,3,5,1])
vocab_size=6
output_dim=3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [14]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [15]:
print(embedding_layer(inputs_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Kodowanie pozycji słów

In [15]:
vocab_size = 100277
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [17]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_lenght=max_length,
    stride=max_length, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Identyfikatory topkenów:\n", inputs)
print("\nKształt danych wejściowych:\n", inputs.shape)

Identyfikatory topkenów:
 tensor([[   57,  8805,  3059,   577],
        [10196,  6077,    64, 75432],
        [ 7762,    64,   480,   285],
        [22464,    64, 15036,  9148],
        [46894, 14531, 63387, 35201],
        [ 4458,  1389,  2665,  7886],
        [ 1940, 62151,   258, 62151],
        [57403,    89,  5979,    86]])

Kształt danych wejściowych:
 torch.Size([8, 4])


In [16]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [18]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [19]:
#Dodanie tensora pozycyjnego do osadzeń wektorów

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
